##**Import Library**

In [ ]:
!pip install newspaper3k
!pip install nlp-
!pip install Sastrawi
from newspaper import Article
from nltk import PorterStemmer
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
import numpy as np

import string
import nltk
import re
nltk.download()
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from gensim.models import FastText
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")

##**Scrapping Data**

### **Liputan 6**

In [ ]:
# Inisialisasi dataframe
df_liputan6 = pd.DataFrame(columns=["URL", "Title", "Media", "Label"])

In [ ]:
# Daftar URL sumber berita dan kategori
news_URL = [
    {"link": "https://www.liputan6.com/news/politik", "category": "Politik", "media": "Liputan 6"},
    {"link": "https://www.liputan6.com/bola", "category": "Olahraga", "media": "Liputan 6"},
    {"link": "https://www.liputan6.com/hot", "category": "Hiburan", "media": "Liputan 6"},
]

# Set seed untuk random.choice
random.seed(42)

# Fungsi melakukan web scraping dari Liputan 6
def scrape_liputan6(link, category, media, n_news):
    news_list = []
    getURL = requests.get(link)
    readURL = BeautifulSoup(getURL.text, "html.parser")
    news = readURL.find_all("h4")  # Selector sesuai struktur situs Liputan 6

    for news in news[:n_news]:
        link = news.find('a')
        if link:
            title = link.text
            news_list.append({"URL": link['href'], "Title": title, "Media": media, "Label": category})
    return news_list

# Mengumpulkan data dari link url Liputan 6
for source in news_URL:
    n_news = 69 # Jumlah berita yang di scraping
    source = scrape_liputan6(source["link"], source["category"], source["media"], n_news)
    df_liputan6 = df_liputan6.append(source, ignore_index=True)

In [ ]:
df_liputan6

,URL,Title,Media,Label
0,https://www.liputan6.com/news/read/5446821/mk-...,MK Gelar Lagi Sidang Batas Usia Capres-Cawapre...,Liputan 6,Politik
1,https://www.liputan6.com/news/read/5446801/saa...,"Saat Puncak HUT ke-59 Golkar, Gus Miftah Bangg...",Liputan 6,Politik
2,https://www.liputan6.com/news/read/5446768/pra...,"Prabowo: Jokowi Bukan Kopassus, Tapi Keberania...",Liputan 6,Politik
3,https://www.liputan6.com/news/read/5446857/mem...,Memahami Pernyataan Djarot Saiful Hidayat Terk...,Liputan 6,Politik
4,https://www.liputan6.com/news/read/5446739/6-h...,6 Hal yang Disampaikan Jokowi saat Hadiri HUT ...,Liputan 6,Politik
...,...,...,...,...
202,https://www.liputan6.com/hot/read/5446477/potr...,"Potret 14 Seleb di Acara Ulang Tahun Ashanty, ...",Liputan 6,Hiburan
203,https://www.liputan6.com/hot/read/5447010/dedi...,"Dedikasikan Untuk Palestina, Konten Makeup Kar...",Liputan 6,Hiburan
204,https://www.liputan6.com/hot/read/5446643/gumi...,"Gumiho adalah Siluman Rubah Ekor Sembilan, Leg...",Liputan 6,Hiburan
205,https://www.liputan6.com/hot/read/5447057/wani...,Wanita Ini Bagikan Momen Bareng Pacar dan Saha...,Liputan 6,Hiburan


*   Melakukan *scraping* pada media `Liputan 6`  dengan *labelling* 3 kategori yang berbeda (politik, olahraga, hiburan) dari keseluruhan data yang ada.
*   Langkah pertama yang saya lakukan adalah membuat dataframe `df_liputan6` yang berisi *link website* berita, judul berita, sumber media, dan *label* berita (kategori berita).
*   Saya membuat list pada variable `news_URL`. List ini berisi dictionary yang memuat *link* berita, kategori, dan media sumber. Selanjutnya, saya membuat *function* `scrape_liputan6()` untuk melakukan *web scraping* dari URL yang ada, mengurai halaman HTML menggunakan BeautifulSoup, dan mengekstrak informasi dari HTML sesuai dengan struktur selektor-nya.
*   Terakhir, saya melakukan *looping* untuk iterasi seluruh data dalam `news_URL` dan memanggil function `scrape_liputan6()` pada tiap iterasi. Hasilnya disimpan pada variable `source` dan ditambahakan pada `df_liputan6`.
*   Saya melakukan *scraping* sebanyak 69 data pada tiap label, sehingga mendapatkan 207 data dengan 4 variable secara keseluruhan.










### **Kompas.com**

In [ ]:
# Inisialisasi dataframe
df_kompas = pd.DataFrame(columns=["URL", "Title", "Media", "Label"])

In [ ]:
# Daftar URL sumber berita dan kategori
news_URL = [
    {"link": "https://nasional.kompas.com/", "category": "Politik", "media": "Kompas.com"},
    {"link": "https://kompas.com/sports", "category": "Olahraga", "media": "Kompas.com"},
    {"link": "https://money.kompas.com/whatsnew", "category": "Hiburan", "media": "Kompas.com"},
]

# Set seed untuk random.choice
random.seed(42)

# Fungsi untuk melakukan web scraping dari Kompas.com
def scrape_kompas(link, category, media, n_news):
    news_list = []
    getURL = requests.get(link)
    readURL = BeautifulSoup(getURL.text, "html.parser")
    news = readURL.find_all("h3")  # Selector sesuai struktur situs Kompas.com

    for news in news[:n_news]:
        link = news.find('a')
        if link:
            title = link.text
            news_list.append({"URL": link['href'], "Title": title, "Media": media, "Label": category})
    return news_list

# Mengumpulkan data dari Kompas.com
for source in news_URL:
    n_news = 22  # Jumlah berita yang di scraping
    source = scrape_kompas(source["link"], source["category"], source["media"], n_news)
    df_kompas = df_kompas.append(source, ignore_index=True)

In [ ]:
df_kompas.head(5)

,URL,Title,Media,Label
0,https://nasional.kompas.com/read/2023/11/07/06...,270 Orang di Barisan Prabowo-Gibran...,Kompas.com,Politik
1,https://nasional.kompas.com/read/2023/11/07/23...,Komisi I DPR Wanti-wanti KSAD Agus agar Netral...,Kompas.com,Politik
2,https://pubads.g.doubleclick.net/gampad/clk?id...,Pemerintah Paparkan Sejumlah Upaya untuk Dukun...,Kompas.com,Politik
3,https://nasional.kompas.com/read/2023/11/07/23...,"Soal Temuan Keripik Pisang Narkoba, Pimpinan K...",Kompas.com,Politik
4,https://nasional.kompas.com/read/2023/11/07/23...,"Terbuki Lakukan Pelanggaran Berat, Anwar Usman...",Kompas.com,Politik


In [ ]:
print("Jumlah baris:", df_kompas.shape[0])
print("Jumlah kolom:", df_kompas.shape[1])

Jumlah baris: 52
Jumlah kolom: 4


*   Melakukan *scraping* pada media `Kompas.com `  dengan *labelling* 3 kategori yang berbeda (politik, olahraga, hiburan) dari keseluruhan data yang ada.
*   Langkah pertama yang saya lakukan adalah membuat dataframe `df_kompas` yang berisi *link website* berita, judul berita, sumber media, dan *label* berita (kategori berita).
*   Saya membuat list pada variable `news_URL`. List ini berisi dictionary yang memuat *link* berita, kategori, dan media sumber. Selanjutnya, saya membuat *function* `scrape_kompas()` untuk melakukan *web scraping* dari URL yang ada, mengurai halaman HTML menggunakan BeautifulSoup, dan mengekstrak informasi dari HTML sesuai dengan struktur selektor-nya.
*   Terakhir, saya melakukan *looping* untuk iterasi seluruh data dalam `news_URL` dan memanggil function `scrape_kompas()` pada tiap iterasi. Hasilnya disimpan pada variable `source` dan ditambahakan pada `df_kompas`.
*   Saya melakukan *scraping* sebanyak 22 data pada tiap label, sehingga seharusnya ada sekitar 66 data. Tetapi, data yang saya dapatkan hanya 52 data. Hal ini disebabkan karena dalam website media tersebut, data dalam kategori politik/hiburan/olahraga sudah tidak ada lagi atau adanya *limit* jumlah data yang bisa di scrapping dari media tsb.














### **Detik.com**


In [ ]:
# Inisialisasi dataframe
df_detik = pd.DataFrame(columns=["URL", "Title", "Media", "Label"])

In [ ]:
# Daftar URL sumber berita dan kategori
news_URL = [
    {"link": "https://news.detik.com/pemilu", "category": "Politik", "media": "Detik.com"},
    {"link": "https://sport.detik.com/", "category": "Olahraga", "media": "Detik.com"},
    {"link": "https://hot.detik.com/", "category": "Hiburan", "media": "Detik.com"},
]

# Set seed untuk random.choice
random.seed(42)

# Fungsi untuk melakukan web scraping dari Detik.com
def scrape_detik(link, category, media, n_news):
    news_list = []
    getURL = requests.get(link)
    readURL = BeautifulSoup(getURL.text, "html.parser")
    news = readURL.find_all("h3")  # Selector sesuai struktur situs Detik.com

    for news in news[:n_news]:
        link = news.find('a')
        if link:
            title = link.text
            news_list.append({"URL": link['href'], "Title": title, "Media": media, "Label": category})
    return news_list

# Mengumpulkan data dari Detik.com
for source in news_URL:
    n_news = 50 # Jumlah berita yang di scraping
    source = scrape_detik(source["link"], source["category"], source["media"], n_news)
    df_detik = df_detik.append(source, ignore_index=True)

In [ ]:
df_detik

,URL,Title,Media,Label
0,https://news.detik.com/pemilu/d-7024613/tkn-mi...,\n\r\n TKN Minta Aparat...,Detik.com,Politik
1,https://news.detik.com/pemilu/d-7024611/saleh-...,\n\r\n Saleh Daulay ke ...,Detik.com,Politik
2,https://news.detik.com/pemilu/d-7024587/ribut-...,\n\r\n Ribut! Masinton ...,Detik.com,Politik
3,https://news.detik.com/pemilu/d-7024577/anies-...,\n\r\n Anies: Kalau Sek...,Detik.com,Politik
4,https://news.detik.com/pemilu/d-7024574/isi-da...,\n\r\n Isi dan Link Dow...,Detik.com,Politik
...,...,...,...,...
135,https://hot.detik.com/celeb/d-7023109/jawaban-...,\n Jawaban Luna Maya soal Hilan...,Detik.com,Hiburan
136,https://hot.detik.com/celeb/d-7023051/blak-bla...,\n Blak-blakan Sudah Pisah dari...,Detik.com,Hiburan
137,https://hot.detik.com/celeb/d-7023035/klarifik...,\n Klarifikasi Fuji Marah-marah...,Detik.com,Hiburan
138,https://hot.detik.com/celeb/d-7021653/azriel-h...,\n Azriel Hermansyah Ungkap Tip...,Detik.com,Hiburan


*   Melakukan *scraping* pada media `Detik.com`  dengan *labelling* 3 kategori yang berbeda (politik, olahraga, hiburan) dari keseluruhan data yang ada.
*   Langkah pertama yang saya lakukan adalah membuat dataframe `df_detik` yang berisi *link website* berita, judul berita, sumber media, dan *label* berita (kategori berita).
*   Saya membuat list pada variable `news_URL`. List ini berisi dictionary yang memuat *link* berita, kategori, dan media sumber. Selanjutnya, saya membuat *function* `scrape_detik()` untuk melakukan *web scraping* dari URL yang ada, mengurai halaman HTML menggunakan BeautifulSoup, dan mengekstrak informasi dari HTML sesuai dengan struktur selektor-nya.
*   Terakhir, saya melakukan *looping* untuk iterasi seluruh data dalam `news_URL` dan memanggil function `scrape_detik()` pada tiap iterasi. Hasilnya disimpan pada variable `source` dan ditambahakan pada `df_detik`.
*   Saya melakukan *scraping* sebanyak 50 data pada tiap label, sehingga seharusnya ada sekitar 150 data. Tetapi, data yang saya dapatkan hanya 140 data. Hal ini disebabkan karena dalam website media tersebut, data dalam kategori politik/hiburan/olahraga sudah tidak ada lagi atau adanya *limit* jumlah data yang bisa di scrapping dari media tsb.










## **Convert Data to CSV**

In [ ]:
df_mediaNews = pd.concat([df_liputan6, df_kompas, df_detik], ignore_index=True)
df_mediaNews

,URL,Title,Media,Label
0,https://www.liputan6.com/news/read/5446821/mk-...,MK Gelar Lagi Sidang Batas Usia Capres-Cawapre...,Liputan 6,Politik
1,https://www.liputan6.com/news/read/5446801/saa...,"Saat Puncak HUT ke-59 Golkar, Gus Miftah Bangg...",Liputan 6,Politik
2,https://www.liputan6.com/news/read/5446768/pra...,"Prabowo: Jokowi Bukan Kopassus, Tapi Keberania...",Liputan 6,Politik
3,https://www.liputan6.com/news/read/5446857/mem...,Memahami Pernyataan Djarot Saiful Hidayat Terk...,Liputan 6,Politik
4,https://www.liputan6.com/news/read/5446739/6-h...,6 Hal yang Disampaikan Jokowi saat Hadiri HUT ...,Liputan 6,Politik
...,...,...,...,...
394,https://hot.detik.com/celeb/d-7023109/jawaban-...,\n Jawaban Luna Maya soal Hilan...,Detik.com,Hiburan
395,https://hot.detik.com/celeb/d-7023051/blak-bla...,\n Blak-blakan Sudah Pisah dari...,Detik.com,Hiburan
396,https://hot.detik.com/celeb/d-7023035/klarifik...,\n Klarifikasi Fuji Marah-marah...,Detik.com,Hiburan
397,https://hot.detik.com/celeb/d-7021653/azriel-h...,\n Azriel Hermansyah Ungkap Tip...,Detik.com,Hiburan


Dataset keseluruhan hasil *scraping* yang dilakukan melalui 3 media dengan 3 label berbeda terdiri dari 399 baris dan 4 kolom.

In [ ]:
for idx, row in df_mediaNews.iterrows():
    link = row['URL']
    news = Article(link)

    try:
        news.download()
        news.parse()
        text = news.text
    except Exception as x:
        print("Failed scraping from " + link + ": " + str(x))
        continue  # Melanjutkan ke URL selanjutnya jika terjadi kesalahan

    # Update the 'Text' column with the scraped text
    df_mediaNews.at[idx, 'Text'] = text

Failed scraping from https://news.detik.com/pemilu/d-7024577/anies-kalau-sekadar-kerja-tanpa-ideologi-ya-tak-ada-arah: Article `download()` failed with HTTPSConnectionPool(host='news.detik.com', port=443): Read timed out. (read timeout=7) on URL https://news.detik.com/pemilu/d-7024577/anies-kalau-sekadar-kerja-tanpa-ideologi-ya-tak-ada-arah
Failed scraping from https://news.detik.com/pemilu/d-7024259/anwar-usman-tak-terbukti-bohong-absen-rph-tunda-bentuk-mkmk-permanen: Article `download()` failed with HTTPSConnectionPool(host='news.detik.com', port=443): Read timed out. (read timeout=7) on URL https://news.detik.com/pemilu/d-7024259/anwar-usman-tak-terbukti-bohong-absen-rph-tunda-bentuk-mkmk-permanen
Failed scraping from https://news.detik.com/pemilu/d-7024060/pertimbangan-mkmk-sanksi-teguran-lisan-6-hakim-mk-putusan-usia-capres-bocor: Article `download()` failed with HTTPSConnectionPool(host='news.detik.com', port=443): Read timed out. (read timeout=7) on URL https://news.detik.com/pe



*   Saya men-download berita menggunakan news.download() dan menguraikan menggunakan news.parse(). Jika berhasil, teks disimpan dalam variabel `text`. Jika tidak berhasil, akan mencetak URL yang tidak berhasil didapatkan teksnya, lalu lanjut ke URL berikutnya.
*   Selanjutnya, memperbarui kolom 'Text' di dataframe dengan teks berita yang berhasil di *scraping*


In [ ]:
df_mediaNews

,URL,Title,Media,Label,Text
0,https://www.liputan6.com/news/read/5446821/mk-...,MK Gelar Lagi Sidang Batas Usia Capres-Cawapre...,Liputan 6,Politik,"Liputan6.com, Jakarta - Mahkamah Konstitusi (M..."
1,https://www.liputan6.com/news/read/5446801/saa...,"Saat Puncak HUT ke-59 Golkar, Gus Miftah Bangg...",Liputan 6,Politik,"Liputan6.com, Jakarta - Partai Golkar menggela..."
2,https://www.liputan6.com/news/read/5446768/pra...,"Prabowo: Jokowi Bukan Kopassus, Tapi Keberania...",Liputan 6,Politik,"Liputan6.com, Jakarta - Menteri Pertahanan Pra..."
3,https://www.liputan6.com/news/read/5446857/mem...,Memahami Pernyataan Djarot Saiful Hidayat Terk...,Liputan 6,Politik,"Liputan6.com, Jakarta - Ketua Umum Yayasan Pen..."
4,https://www.liputan6.com/news/read/5446739/6-h...,6 Hal yang Disampaikan Jokowi saat Hadiri HUT ...,Liputan 6,Politik,"Liputan6.com, Jakarta - Presiden Joko Widodo a..."
...,...,...,...,...,...
394,https://hot.detik.com/celeb/d-7023109/jawaban-...,\n Jawaban Luna Maya soal Hilan...,Detik.com,Hiburan,Luna Maya dan Maxime Bouttier belum lama ini m...
395,https://hot.detik.com/celeb/d-7023051/blak-bla...,\n Blak-blakan Sudah Pisah dari...,Detik.com,Hiburan,Musisi Ardhito Pramono secara mengejutkan memb...
396,https://hot.detik.com/celeb/d-7023035/klarifik...,\n Klarifikasi Fuji Marah-marah...,Detik.com,Hiburan,Sikap Fuji kepada karyawan menjadi sorotan pub...
397,https://hot.detik.com/celeb/d-7021653/azriel-h...,\n Azriel Hermansyah Ungkap Tip...,Detik.com,Hiburan,"Anak artis Anang Hermansyah, Azriel Hermansyah..."


In [ ]:
df_mediaNews.to_csv("media_news.csv", index=False)

Saya menyimpan dataframe `df_mediaNews` yang berisi link berita, judul berita, media, label, dan text berita ke dalam bentuk csv file.

## **Read CSV File**



In [ ]:
df_news = pd.read_csv('https://raw.githubusercontent.com/jasminemutia/dataset/main/media_news.csv')

In [ ]:
df_news.head()

,URL,Title,Media,Label,Text
0,https://www.liputan6.com/news/read/5446821/mk-...,MK Gelar Lagi Sidang Batas Usia Capres-Cawapre...,Liputan 6,Politik,"Liputan6.com, Jakarta - Mahkamah Konstitusi (M..."
1,https://www.liputan6.com/news/read/5446801/saa...,"Saat Puncak HUT ke-59 Golkar, Gus Miftah Bangg...",Liputan 6,Politik,"Liputan6.com, Jakarta - Partai Golkar menggela..."
2,https://www.liputan6.com/news/read/5446768/pra...,"Prabowo: Jokowi Bukan Kopassus, Tapi Keberania...",Liputan 6,Politik,"Liputan6.com, Jakarta - Menteri Pertahanan Pra..."
3,https://www.liputan6.com/news/read/5446857/mem...,Memahami Pernyataan Djarot Saiful Hidayat Terk...,Liputan 6,Politik,"Liputan6.com, Jakarta - Ketua Umum Yayasan Pen..."
4,https://www.liputan6.com/news/read/5446739/6-h...,6 Hal yang Disampaikan Jokowi saat Hadiri HUT ...,Liputan 6,Politik,"Liputan6.com, Jakarta - Presiden Joko Widodo a..."


In [ ]:
df_news = df_news.drop(["URL", "Title"], axis=1)

Menghapus kolom `URL` dan `Title` untuk menyesuaikan format pada soal yaitu kolom yang berisi `Text`, `Media`, dan `Label`.

In [ ]:
df_news = df_news[["Text", "Media", "Label"]]

In [ ]:
df_news.head(3)

,Text,Media,Label
0,"Liputan6.com, Jakarta - Mahkamah Konstitusi (M...",Liputan 6,Politik
1,"Liputan6.com, Jakarta - Partai Golkar menggela...",Liputan 6,Politik
2,"Liputan6.com, Jakarta - Menteri Pertahanan Pra...",Liputan 6,Politik


##  **Data Cleansing**

In [ ]:
news_text = df_news[['Text','Label']].copy()

### **Case Folding**

In [ ]:
news_text['Text'] = news_text['Text'].str.lower()

In [ ]:
news_text.head(3)

,Text,Label
0,"liputan6.com, jakarta - mahkamah konstitusi (m...",Politik
1,"liputan6.com, jakarta - partai golkar menggela...",Politik
2,"liputan6.com, jakarta - menteri pertahanan pra...",Politik




*   Langkah pertama yang dilakukan yaitu mengubah seluruh huruf dalam teks berita menjadi huruf kecil menggunakan `str.lower()`.
*   Hal ini dilakukan untuk menghindari kondisi dimana kata yang sebenarnya sama tetapi dianggap berbeda karena perbedaan huruf besar dan kecil.

### **Remove Number, "https", ".com", Symbol, Enter, and  Multiple Whitespace**

In [ ]:
# Remove Number
news_text['Text'] = news_text['Text'].str.replace(r'\d+', '', regex=True)

# Remove "https" and ".com"
news_text['Text'] = news_text['Text'].str.replace(r'\.com|\.id|\.co', '', regex=True)
news_text['Text'] = news_text['Text'].str.replace(r'https\S+|http\S+|www\.\S+', '', regex=True)

# Remove Enter
news_text['Text'] = news_text['Text'].str.replace(r'\n', ' ', regex=True)

*   Menghapus seluruh angka yang ada di dalam teks untuk memudahkan proses analisis selanjutnya. Dalam teks berita sendiri, angka tidak terlalu memiliki makna khusus.
*   Menghapus *link website* dari teks. *Link website* seringkali tidak memberikan informasi yang penting dalam analisis teks. Hal ini dilakukan untuk membantu fokus pada konten teks aktual.
*   Mengganti karakter `newline` atau enter dengan spasi. Ini membantu mencegah pemisahan kata karena karakter `newline`.






In [ ]:
# punctuation yang dihilangkan
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Symbol yang ingin dihilangkan dalam teks secara keseluruhan. Di dalamnya sudah termasuk *remove hashtag*.

In [ ]:
# Remove punctuation from the 'text' column
news_text['Text'] = news_text['Text'].str.replace('[{}]'.format(string.punctuation), '', regex=True)

Menghapus symbol dari *text* tersebut bertujuan untuk lebih fokus ke teks. Biasanya symbol tidak memberikan nilai signifikan dalam *text* analisis.

In [ ]:
# Remove multiple whitespcae
def remove_whitespace(text):
    if isinstance(text, str):
        return re.sub(r'\s+', ' ', text)
    else:
        return text

news_text['Text'] = news_text['Text'].apply(remove_whitespace)

Menghapus spasi berlebihan untuk menjaga teks data tetap bersih dan lebih mudah untuk diolah.

In [ ]:
news_text.head(3)

,Text,Label
0,liputan jakarta mahkamah konstitusi mk kembali...,Politik
1,liputan jakarta partai golkar menggelar puncak...,Politik
2,liputan jakarta menteri pertahanan prabowo sub...,Politik


Teks data sudah dibersikan dengan mengganti ke *lowercase*, menghapus angka, *https*, *.com*, *enter*, *symbol*, dan *multiple whitespace*.

## **Data Splitting**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(news_text['Text'], news_text['Label'], test_size = 0.2, random_state = 32,stratify=news_text['Label'])

In [ ]:
print('Train set: {} baris'.format(X_train.shape[0]))
print('Test set: {} baris'.format(X_test.shape[0]))

Train set: 319 baris
Test set: 80 baris


Output diatas menunjukkan jumlah data setelah dibagi menjadi 2 dengan proporsi 80% training set dan 20% test set.

In [ ]:
X_train.head(3)

217    jakarta kompas menteri koordinator bidang poli...
368    bagi kamu penggemar one piece disebut nakama s...
29     liputan jakarta komisi pemberantasan korupsi k...
Name: Text, dtype: object

In [ ]:
y_test.head(3)

321    Olahraga
76     Olahraga
30      Politik
Name: Label, dtype: object

##**Data Preprocessing**

###**Tokenization and Stop Words Removals**

In [ ]:
# Ambil daftar stop words dalam bahasa Indonesia
stop_words = set(stopwords.words('indonesian'))

# Fungsi untuk melakukan tokenisasi dan membersihkan stop words
def tokenize_removeStopwords(text):
    if isinstance(text, str):
        # Tokenisasi kata
        wordToken = word_tokenize(text)
        # Hapus stop words dan tanda baca
        wordToken = [word.lower() for word in wordToken if (word.isalpha() and word.lower() not in stop_words)]
        return wordToken
    else:
        return []

# Terapkan fungsi tokenisasi
X_train = X_train.apply(tokenize_removeStopwords)
X_test = X_test.apply(tokenize_removeStopwords)



*   Melakukan *tokenization* yaitu memecah teks menjadi unit-unit kecil yang disebut sebagai token/kata pada `X_train` dan `X_test`.
*   Menghapus *stopwords* yaitu kata-kata umum yang sering muncul tetapi biasanya tidak memberikan informasi tambahan yang signifikan.



In [ ]:
X_train.head(3)

217    [jakarta, kompas, menteri, koordinator, bidang...
368    [penggemar, one, piece, nakama, menyambangi, o...
29     [liputan, jakarta, komisi, pemberantasan, koru...
Name: Text, dtype: object

In [ ]:
X_test.head(3)

321    [marcus, rashford, paceklik, gol, liga, champi...
76     [liputan, jakarta, kapten, timnas, indonesia, ...
30     [liputan, jakarta, pengamat, politik, universi...
Name: Text, dtype: object

## **Stemming**

In [ ]:
fc = StemmerFactory()
stem = fc.create_stemmer()

In [ ]:
# Stemming pada setiap kata
X_trainStemming = [[stem.stem(word) for word in text] for text in X_train]
X_testStemming = [[stem.stem(word) for word in text] for text in X_test]
print(X_trainStemming[0])
print(X_testStemming[0])

['jakarta', 'kompas', 'menteri', 'koordinator', 'bidang', 'politik', 'hukum', 'aman', 'calon', 'wakil', 'presiden', 'mahfud', 'md', 'aku', 'bangga', 'mahkamah', 'konstitusi', 'mk', 'pascaputusan', 'majelis', 'hormat', 'mk', 'mkmk', 'kait', 'langgar', 'etik', 'hakim', 'mk', 'mahfud', 'sedih', 'malu', 'status', 'mantan', 'hakim', 'ketua', 'mk', 'sedih', 'malu', 'hakim', 'ketua', 'mk', 'mkmk', 'keluar', 'putus', 'langgar', 'etik', 'hakim', 'konstitusi', 'bangga', 'mk', 'guardian', 'of', 'constitution', 'tulis', 'mahfud', 'akun', 'twitternya', 'mohmahfudmd', 'selasa', 'baca', 'mkmk', 'henti', 'anwar', 'usman', 'jubir', 'anies', 'tantang', 'prabowo', 'ganti', 'cawapres', 'kompas', 'izin', 'staf', 'khusus', 'menko', 'polhukam', 'rizal', 'musrary', 'kutip', 'nyata', 'mahfud', 'tweetnya', 'mahfud', 'salam', 'hormat', 'anggota', 'mkmk', 'jimly', 'asshiddiqie', 'bintan', 'saragih', 'wahihuddin', 'adams', 'salam', 'hormat', 'jimly', 'bintan', 'wahiduddin', 'tulis', 'mkmk', 'putus', 'henti', 'anwa

Stemming dilakukan untuk mengubah kata ke bentuk dasarnya dengan cara menghilangkan awalan dan akhiran dalam bahasa Indonesia. Tujuannya untuk mengurangi variasi kata ke bentuk dasar agar lebih mudah dalam analisis teks.

## **Text Representation**

### **Word2Vec - Skipgram**

In [ ]:
# Create Word2Vec model (Skip-gram)
skipgramModel = Word2Vec(sentences=X_trainStemming, vector_size=50, window=5, sg=1, min_count=3)

In [ ]:
# Function to get the vector representation
def vectorRepresentation(text, model):
    vector = [model.wv[text] for text in text if text in model.wv]
    return sum(vector)/len(vector) if vector else [0] * model.vector_size

*   Membuat model Word2Vec (Skipgram) menggunakan *training data* yang telah melalui proses tokenization dan stemming.
*   Menggunakan vector size = 50 dan hanya mempertimbangkan kata-kata yang muncul setidaknya sebanyak 3 kali.
*   `window=5` berarti jumlah kata yang diperhitungkan di sekitar target sebanyak.`sg=1` menunjukkan bahwa menggunakan metode Skipgram.
*   `trainVectors` dan `testVectors` menghasilkan representasi vektor untuk setiap text dalam training set menggunakan model Word2Vec yang telah dibuat.







In [ ]:
# Get vector representations for the training and test data
trainVectors = [vectorRepresentation(text, skipgramModel) for text in X_trainStemming]
testVectors = [vectorRepresentation(text, skipgramModel) for text in X_testStemming]

# Convert the lists to arrays for SVM and Random Forest
X_train_skipgram = np.array(trainVectors)
X_test_skipgram = np.array(testVectors)

### **Fasttext**

In [ ]:
# Create FastText model
fasttextModel = FastText(sentences=X_trainStemming, vector_size=50, window=5, min_count=3, sg=1)

# Get vector representations for training data
trainVectors1 = [vectorRepresentation(j, fasttextModel) for j in X_trainStemming]
testVectors1 = [vectorRepresentation(j, fasttextModel) for j in X_testStemming]

# Convert the lists to arrays for SVM and Random Forest
X_train_ff = np.array(trainVectors1)
X_test_ff = np.array(testVectors1)

*   Membuat model Fasttext menggunakan *training data* yang telah melalui proses tokenization dan stemming.
*   Menggunakan vector size = 50 dan hanya mempertimbangkan kata-kata yang muncul setidaknya sebanyak 3 kali.
*   `window=5` berarti jumlah kata yang diperhitungkan di sekitar target sebanyak.`sg=1` menunjukkan bahwa menggunakan metode Skipgram.
*   `trainVectors` dan `testVectors` menghasilkan representasi vektor untuk setiap text dalam training set menggunakan model Word2Vec yang telah dibuat.







## **Modelling**

### **SVM**

In [ ]:
# Train SVM classifier
SVM1 = SVC()
SVM1.fit(X_train_skipgram, y_train)

# Make predictions using SVM
SVM1_prediction = SVM1.predict(X_test_skipgram)

# Evaluate SVM performance
SVM_accuracy = accuracy_score(y_test, SVM1_prediction)
SVM_report = classification_report(y_test, SVM1_prediction)

In [ ]:
# Print the results
print("SVM (Skipgram) Accuracy :", SVM_accuracy)
print("SVM (Skipgram) Classification Report :\n", SVM_report)

SVM (Skipgram) Accuracy : 0.9125
SVM (Skipgram) Classification Report :
               precision    recall  f1-score   support

     Hiburan       0.83      0.96      0.89        26
    Olahraga       1.00      0.89      0.94        28
     Politik       0.92      0.88      0.90        26

    accuracy                           0.91        80
   macro avg       0.92      0.91      0.91        80
weighted avg       0.92      0.91      0.91        80



Output diatas menunjukkan akurasi dari model SVM menggunakan *text representation* Skipgram adalah 91%. Ini berarti 91% dari label kelas pada testing set diprediksi dengan benar oleh model. Penggunaan metode skipgram dalam SVM memiliki akurasi yang lebih tinggi dibandingkan Fasttext.



In [ ]:
# Train SVM classifier
SVM2 = SVC()
SVM2.fit(X_train_ff, y_train)

# Make predictions using SVM
SVM2_prediction = SVM2.predict(X_test_ff)

# Evaluate SVM performance
SVM2_accuracy = accuracy_score(y_test, SVM2_prediction)
SVM2_report = classification_report(y_test, SVM2_prediction)

In [ ]:
# Print the results
print("SVM (Fasttext) Accuracy :", SVM2_accuracy)
print("SVM (Fasttext) Classification Report :\n", SVM2_report)

SVM (Fasttext) Accuracy : 0.8875
SVM (Fasttext) Classification Report :
               precision    recall  f1-score   support

     Hiburan       0.74      1.00      0.85        26
    Olahraga       1.00      0.89      0.94        28
     Politik       1.00      0.77      0.87        26

    accuracy                           0.89        80
   macro avg       0.91      0.89      0.89        80
weighted avg       0.92      0.89      0.89        80



Output diatas menunjukkan akurasi dari model SVM menggunakan *text representation* Fasttext adalah 89%. Ini berarti 89% dari label kelas pada testing set diprediksi dengan benar oleh model.



## **Random Forest**

In [ ]:
# Train Random Forest classifier
RF1 = RandomForestClassifier()
RF1.fit(X_train_skipgram, y_train)

# Make predictions using Random Forest
RF1_predictions = RF1.predict(X_test_skipgram)

# Evaluate Random Forest performance
RF1_accuracy = accuracy_score(y_test, RF1_predictions)
RF1_report = classification_report(y_test, RF1_predictions)

In [ ]:
# Print the results
print("Random Forest (Skipgram) Accuracy:", RF1_accuracy)
print("Random Forest (Skipgram) Classification Report:\n", RF1_report)

Random Forest (Skipgram) Accuracy: 0.9375
Random Forest (Skipgram) Classification Report:
               precision    recall  f1-score   support

     Hiburan       0.92      0.92      0.92        26
    Olahraga       1.00      0.89      0.94        28
     Politik       0.90      1.00      0.95        26

    accuracy                           0.94        80
   macro avg       0.94      0.94      0.94        80
weighted avg       0.94      0.94      0.94        80



Output diatas menunjukkan akurasi dari model Random Forest menggunakan *text representation* Skipgram adalah 94%. Ini berarti 94% dari label kelas pada testing set diprediksi dengan benar oleh model. Penggunaan metode random forest dalam SVM memiliki akurasi yang lebih tinggi dibandingkan Fasttext.



In [ ]:
# Train Random Forest classifier
RF2 = RandomForestClassifier()
RF2.fit(X_train_ff, y_train)

# Make predictions using Random Forest
RF2_predictions = RF2.predict(X_test_ff)

# Evaluate Random Forest performance
RF2_accuracy = accuracy_score(y_test, RF2_predictions)
RF2_report = classification_report(y_test, RF2_predictions)

In [ ]:
# Print the results
print("Random Forest (Fasttext) Accuracy:", RF2_accuracy)
print("Random Forest (Fasttext) Classification Report:\n", RF2_report)

Random Forest (Fasttext) Accuracy: 0.925
Random Forest (Fasttext) Classification Report:
               precision    recall  f1-score   support

     Hiburan       0.86      0.96      0.91        26
    Olahraga       1.00      0.86      0.92        28
     Politik       0.93      0.96      0.94        26

    accuracy                           0.93        80
   macro avg       0.93      0.93      0.93        80
weighted avg       0.93      0.93      0.93        80



Output diatas menunjukkan akurasi dari model Random Forest menggunakan *text representation* Fasttext adalah 93%. Ini berarti 93% dari label kelas pada testing set diprediksi dengan benar oleh model.



**Kesimpulan :** Penggunaan skipgram pada metode Random Forest maupun SVM memiliki akurasi yang lebih baik. Sedangkan modeling menggunakan random forest menghasilkan akurasi yang lebih tinggi dengan text representation skipgram maupun fasttext.